In [1]:
# banks_project.py

import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime


# -------------------------
# CONFIG (per project spec)
# -------------------------
DATA_URL = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
EXCHANGE_RATE_CSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"

TABLE_ATTRS_EXTRACT = ["Name", "MC_USD_Billion"]
TABLE_ATTRS_FINAL = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]

OUTPUT_CSV = "./Largest_banks_data.csv"
DB_NAME = "Banks.db"
TABLE_NAME = "Largest_banks"
LOG_FILE = "code_log.txt"


# -------------------------
# TASK 1: Logging
# -------------------------
def log_progress(message: str, log_file: str = LOG_FILE) -> None:
    """
    Appends a timestamped message to the log file.
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] {message}\n")


# -------------------------
# TASK 2: Extraction
# -------------------------
def extract(url: str) -> pd.DataFrame:
    """
    Extracts the 'By market capitalization' table from the provided URL and returns
    a DataFrame containing: Name, MC_USD_Billion (top 10 rows).
    """
    response = requests.get(url, timeout=30)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # Find the heading span with id="By_market_capitalization"
    heading = soup.find(id="By_market_capitalization")
    if heading is None:
        raise ValueError("Could not find the 'By market capitalization' section (id=By_market_capitalization).")

    # The target table is typically the first wikitable after the heading
    table = heading.find_parent(["h2", "h3"]).find_next("table", class_="wikitable")
    if table is None:
        raise ValueError("Could not find the wikitable following the 'By market capitalization' heading.")

    # Parse table rows
    rows = table.find_all("tr")
    data = []

    for r in rows[1:]:  # skip header
        cols = r.find_all(["th", "td"])
        if len(cols) < 3:
            continue

        # Typical columns (as of the archived page):
        # 0: Rank, 1: Bank name, 2: Market cap (US$ billion)
        name = cols[1].get_text(strip=True)
        mc_text = cols[2].get_text(strip=True).replace(",", "")
        try:
            mc_usd = float(mc_text)
        except ValueError:
            continue

        data.append([name, mc_usd])

        if len(data) == 10:
            break

    df = pd.DataFrame(data, columns=TABLE_ATTRS_EXTRACT)
    return df


# -------------------------
# TASK 3: Transformation
# -------------------------
def transform(df: pd.DataFrame, exchange_rate_csv: str) -> pd.DataFrame:
    """
    Adds MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion using exchange rates from CSV,
    rounded to 2 decimals.
    """
    rates_df = pd.read_csv(exchange_rate_csv)

    # Expecting columns like: Currency, Rate
    # Build a dict: {"GBP": rate, "EUR": rate, "INR": rate}
    rate_map = dict(zip(rates_df["Currency"], rates_df["Rate"]))

    for cur in ["GBP", "EUR", "INR"]:
        if cur not in rate_map:
            raise ValueError(f"Exchange rate for {cur} not found in exchange rate CSV.")

    df_out = df.copy()
    df_out["MC_GBP_Billion"] = (df_out["MC_USD_Billion"] * rate_map["GBP"]).round(2)
    df_out["MC_EUR_Billion"] = (df_out["MC_USD_Billion"] * rate_map["EUR"]).round(2)
    df_out["MC_INR_Billion"] = (df_out["MC_USD_Billion"] * rate_map["INR"]).round(2)

    return df_out[TABLE_ATTRS_FINAL]


# -------------------------
# TASK 4: Load to CSV
# -------------------------
def load_to_csv(df: pd.DataFrame, output_path: str) -> None:
    df.to_csv(output_path, index=False)


# -------------------------
# TASK 5: Load to DB
# -------------------------
def load_to_db(df: pd.DataFrame, db_name: str, table_name: str) -> None:
    conn = sqlite3.connect(db_name)
    try:
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    finally:
        conn.close()


# -------------------------
# TASK 6: Run queries
# -------------------------
def run_queries(db_name: str, table_name: str) -> None:
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()

        queries = [
            f"SELECT * FROM {table_name};",
            f"SELECT AVG(MC_GBP_Billion) FROM {table_name};",
            f"SELECT Name FROM {table_name} LIMIT 5;"
        ]

        for q in queries:
            log_progress(f"Running query: {q}")
            print("\nQUERY:", q)
            cursor.execute(q)
            results = cursor.fetchall()
            for row in results:
                print(row)

    finally:
        conn.close()


# -------------------------
# TASK 1–7: Orchestration
# -------------------------
def main():
    log_progress("Preliminaries complete. Starting ETL process.")

    # Extract
    df_extracted = extract(DATA_URL)
    log_progress("Data extraction complete.")
    print("Extracted DataFrame (top 10):")
    print(df_extracted)

    # Transform
    df_transformed = transform(df_extracted, EXCHANGE_RATE_CSV)
    log_progress("Data transformation complete.")
    print("\nTransformed DataFrame:")
    print(df_transformed)

    # Load to CSV
    load_to_csv(df_transformed, OUTPUT_CSV)
    log_progress("Data saved to CSV file.")
    print(f"\nSaved CSV to: {OUTPUT_CSV}")

    # Load to DB
    load_to_db(df_transformed, DB_NAME, TABLE_NAME)
    log_progress("Data loaded to Database as a table.")
    print(f"Loaded table '{TABLE_NAME}' into database '{DB_NAME}'")

    # Queries
    run_queries(DB_NAME, TABLE_NAME)

    log_progress("Process Complete.")

    # Task 7: Show log contents (verify log entries)
    print("\n--- LOG FILE CONTENTS ---")
    with open(LOG_FILE, "r", encoding="utf-8") as f:
        print(f.read())


if __name__ == "__main__":
    main()


Extracted DataFrame (top 10):
                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81

Transformed DataFrame:
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56     